In [1]:
from datetime import datetime, timedelta

from multiprocessing import Process, Queue, Array

import cartopy.crs as ccrs
import cartopy.feature as cfeature
import cartopy.io.shapereader as shpreader
import matplotlib.pyplot as plt
import metpy.calc as mpcalc
import numpy as np

import requests
import sys

from metpy.plots import StationPlot
from metpy.units import units
from netCDF4 import Dataset, num2date
from scipy.ndimage import gaussian_filter
from siphon.catalog import TDSCatalog
from xarray.backends import NetCDF4DataStore
import xarray as xr

In [3]:
#study date
year = 2020
month = 8
day = 14
hour = 20

dts = [datetime(year,month,day,hour)+timedelta(hours=i) for i in range(7)]

n_proc = 20
debug = 1
queue = Queue()

reader = shpreader.Reader('/users/joshuanielsen/Documents/GitHub/Non_Thesis_Project/Processing_Scripts/county_shape_file/countyl010g.shp')
counties = list(reader.geometries())
COUNTIES = cfeature.ShapelyFeature(counties,ccrs.PlateCarree())


In [4]:
def main():
    for dt in dts:
        ncss = get_dataset(dt)
        query = ncss.query()
        query.lonlat_box(north=50,south=40,east=-90,west=-110)
        query.all_times()
        query.add_lonlat()
        query.accept('netcdf')
        query.variables('Geopotential_height_isobaric',
                        'Relative_humidity_isobaric',
                        'u-component_of_wind_isobaric',
                        'u-component_of_wind_height_above_ground',
                        'v-component_of_wind_isobaric',
                        'v-component_of_wind_height_above_ground',
                        'Temperature_isobaric',
                        'MSLP_MAPS_System_Reduction_msl',
                        'Convective_available_potential_energy_surface',
                        'Convective_inhibition_surface')
    
        data = ncss.get_data(query)
        ds = xr.open_dataset(NetCDF4DataStore(data)).metpy.parse_cf()

        mslp = ds.MSLP_MAPS_System_Reduction_msl[0]
        temp = ds.Temperature_isobaric[0]
        rh = ds.Relative_humidity_isobaric[0]
        height = ds.Geopotential_height_isobaric[0]
        u = ds['u-component_of_wind_isobaric'][0]
        v = ds['v-component_of_wind_isobaric'][0]
        u_wind_10m = np.array(ds['u-component_of_wind_height_above_ground'][0][0].values)
        v_wind_10m = np.array(ds['v-component_of_wind_height_above_ground'][0][0].values)
        hgt = ds['Geopotential_height_isobaric'][0]
    
        CAPE = ds['Convective_available_potential_energy_surface'][0]
        CIN = ds['Convective_inhibition_surface'][0]
    
        common_levels = np.intersect1d(temp.metpy.vertical, u.metpy.vertical)
        temp = temp.metpy.sel(vertical=common_levels)
        rh = rh.metpy.sel(vertical=common_levels)
        u = u.metpy.sel(vertical=common_levels)
        v = v.metpy.sel(vertical=common_levels)
        press = temp.metpy.vertical.metpy.sel(vertical=common_levels)
        dpt = mpcalc.dewpoint_from_relative_humidity(temp,rh)
        hgt = hgt.metpy.sel(vertical=common_levels)

        #smooth mslp a bit
        smooth_mslp = gaussian_filter(mslp,sigma=2,order=0)/100.0 * units("hectopascals")

        #create memory pointers for variables
        if debug > 0:
            print('Creating memory pointers for mixed-layer variables')
        total_points = len(CAPE)*len(CAPE[0])
        this_MLCAPE = Array('d',total_points)
        this_MLCIN = Array('d',total_points)
        this_ML_TEMP = Array('d',total_points)
        this_ML_DPT = Array('d',total_points)
        this_ML_LCL = Array('d',total_points)
        this_ML_LCL_Z = Array('d',total_points)
        this_SMV_R = Array('d',total_points*2)
        this_SMV_L = Array('d',total_points*2)
        this_SMV_MEAN = Array('d',total_points*2)
        this_SHEAR6 = Array('d',total_points*2)
        this_SHEAR3 = Array('d',total_points*2)
        this_EFF_SHEAR = Array('d',total_points*2)
        this_SRH3 = Array('d',total_points)
        this_SRH1 = Array('d',total_points)
        this_MLCAPE = np.frombuffer(this_MLCAPE.get_obj()).reshape((len(CAPE),len(CAPE[0])))
        this_MLCIN = np.frombuffer(this_MLCIN.get_obj()).reshape((len(CAPE),len(CAPE[0])))
        this_ML_TEMP = np.frombuffer(this_ML_TEMP.get_obj()).reshape((len(CAPE),len(CAPE[0])))
        this_ML_DPT = np.frombuffer(this_ML_DPT.get_obj()).reshape((len(CAPE),len(CAPE[0])))
        this_ML_LCL = np.frombuffer(this_ML_LCL.get_obj()).reshape((len(CAPE),len(CAPE[0])))
        this_ML_LCL_Z = np.frombuffer(this_ML_LCL_Z.get_obj()).reshape((len(CAPE),len(CAPE[0])))
        this_SMV_R = np.frombuffer(this_SMV_R.get_obj()).reshape((len(CAPE),len(CAPE[0]),2))
        this_SMV_L = np.frombuffer(this_SMV_L.get_obj()).reshape((len(CAPE),len(CAPE[0]),2))
        this_SMV_MEAN = np.frombuffer(this_SMV_MEAN.get_obj()).reshape((len(CAPE),len(CAPE[0]),2))
        this_SHEAR6 = np.frombuffer(this_SHEAR6.get_obj()).reshape((len(CAPE),len(CAPE[0]),2))
        this_SHEAR3 = np.frombuffer(this_SHEAR3.get_obj()).reshape((len(CAPE),len(CAPE[0]),2))
        this_EFF_SHEAR = np.frombuffer(this_EFF_SHEAR.get_obj()).reshape((len(CAPE),len(CAPE[0]),2))
        this_SRH3 = np.frombuffer(this_SRH3.get_obj()).reshape((len(CAPE),len(CAPE[0])))
        this_SRH1 = np.frombuffer(this_SRH1.get_obj()).reshape((len(CAPE),len(CAPE[0])))

        processes = list()

        if debug > 0:
            print('Calculating thermodynamic values')

        #fill queue with i,j indices
        for i in range(len(CAPE)):
            for j in range(len(CAPE[i])):
                queue.put([i,j])

        for proc in range(n_proc):
            process = Process(target=mixed_layer,args=(this_MLCAPE,this_MLCIN,this_ML_TEMP,this_ML_DPT,this_ML_LCL,this_ML_LCL_Z,press,temp,dpt))
            process.start()
            processes.append(process)
            queue.put(None)
        for process in processes:
            process.join()

        if debug > 0:
            print('Calculating shear values')

        #fill queue with i,j indices
        for i in range(len(CAPE)):
            for j in range(len(CAPE[i])):
                queue.put([i,j])

        for proc in range(n_proc):
            process = Process(target=shear,args=(this_SMV_R,this_SMV_L,this_SMV_MEAN,this_SHEAR6,this_SHEAR3,this_EFF_SHEAR,this_SRH3,this_SRH1,press,u,v,hgt))
            process.start()
            processes.append(process)
            queue.put(None)
        for process in processes:
            process.join()


        MLCAPE = np.array(this_MLCAPE).astype(float)*units("J/kg")
        MLCAPE[MLCAPE.magnitude<=0] = np.nan
        MLCIN = this_MLCIN*units("J/kg")
        MLCIN[MLCIN.magnitude>=0] = np.nan
        ML_TEMP = this_ML_TEMP*units("K")
        ML_DPT = this_ML_DPT*units("K")
        ML_LCL = this_ML_LCL/100.*units("hectopascals")
        ML_LCL_Z = this_ML_LCL_Z*units("meters")
        SHEAR6_TOT = ((this_SHEAR6[:,:,0]**2.0 + this_SHEAR6[:,:,1]**2.0)**0.5) * 1.94384 * units('knots')
        SHEAR3_TOT = ((this_SHEAR3[:,:,0]**2.0 + this_SHEAR3[:,:,1]**2.0)**0.5) * 1.94384 * units('knots')
        EFF_SHEAR_TOT = ((this_EFF_SHEAR[:,:,0]**2.0 + this_EFF_SHEAR[:,:,1]**2.0)**0.5) * 1.94384 * units('knots')
        SRH3 = this_SRH3 #* units("m2/s2")
        print(SRH3)
        SRH1 = this_SRH1 * units("m2/s2")

        if debug > 0:
            print('Creating Plots')

        proj = ccrs.LambertConformal(central_longitude=-95,central_latitude=35,standard_parallels=[35])
        data_proj = ccrs.PlateCarree()
        lon = ds.lon
        lat = ds.lat

        fig=plt.figure(figsize=(10,9))
        ax = fig.add_subplot(1,1,1,projection=proj)
        ax.add_feature(cfeature.STATES,edgecolor='black',linewidth=2,zorder=2)
        ax.add_feature(cfeature.COASTLINE,edgecolor='black',linewidth=2,zorder=2)
        ax.add_feature(COUNTIES,facecolor='none',edgecolor='gray',zorder=1)
        ax.set_extent((-108,-92,41,49))
        cf = ax.pcolormesh(lon,lat,MLCAPE,transform=data_proj,cmap='jet')
        cf.set_clim(0.5,4000)
        cbar = plt.colorbar(cf,ax=ax,orientation='horizontal',pad=0.01,fraction=0.049)
        cbar.ax.set_xlabel('MIXED LAYER CAPE (J/kg)')
        cs = ax.contour(lon,lat,smooth_mslp,np.arange(0,1200,4),colors='black',linewidths=1.5,linestyles='solid',transform=data_proj)
        plt.clabel(cs, fontsize=10, inline=1, inline_spacing=10, fmt="%i", rightside_up=True, use_clabeltext=True)
        ax.barbs(lon.values,lat.values,u_wind_10m*1.94384,v_wind_10m*1.94384,length=6,regrid_shape=20,pivot='middle',transform=data_proj,zorder=10)
        fig.tight_layout(rect=[0,0.01,1,0.98])
        plt.savefig('./%s_MLCAPE.png'%dt.strftime('%Y%m%d_%H'))
        plt.close()

        fig=plt.figure(figsize=(10,9))
        ax = fig.add_subplot(1,1,1,projection=proj)
        ax.add_feature(cfeature.STATES,edgecolor='black',linewidth=2,zorder=2)
        ax.add_feature(cfeature.COASTLINE,edgecolor='black',linewidth=2,zorder=2)
        ax.add_feature(COUNTIES,facecolor='none',edgecolor='gray',linewidth=1,zorder=1)
        ax.set_extent((-102,-93,44,48))
        cf = ax.pcolormesh(lon,lat,MLCAPE,transform=data_proj,cmap='jet')
        cf.set_clim(0.5,4000)
        cbar = plt.colorbar(cf,ax=ax,orientation='horizontal',pad=0.01,fraction=0.049)
        cbar.ax.set_xlabel('MIXED LAYER CAPE (J/kg)')
        cs = ax.contour(lon,lat,smooth_mslp,np.arange(0,1200,4),colors='black',linewidths=1.5,linestyles='solid',transform=data_proj)
        plt.clabel(cs, fontsize=10, inline=1, inline_spacing=10, fmt="%i", rightside_up=True, use_clabeltext=True)
        ax.barbs(lon.values,lat.values,u_wind_10m*1.94384,v_wind_10m*1.94384,length=6,regrid_shape=20,pivot='middle',transform=data_proj,zorder=10)
        fig.tight_layout(rect=[0,0.01,1,0.98])
        plt.savefig('./%s_zoomed_MLCAPE.png'%dt.strftime('%Y%m%d_%H'))
        plt.close()

        fig=plt.figure(figsize=(10,9))
        ax = fig.add_subplot(1,1,1,projection=proj)
        ax.add_feature(cfeature.STATES,edgecolor='black',linewidth=2,zorder=2)
        ax.add_feature(cfeature.COASTLINE,edgecolor='black',linewidth=2,zorder=2)
        ax.add_feature(COUNTIES,facecolor='none',edgecolor='gray',zorder=1)
        ax.set_extent((-108,-92,41,49))
        cf = ax.pcolormesh(lon,lat,ML_LCL,transform=data_proj,cmap='jet_r')
        cf.set_clim(1000,700)
        cbar = plt.colorbar(cf,ax=ax,orientation='horizontal',pad=0.01,fraction=0.049)
        cbar.ax.set_xlabel('MIXED LAYER LCL (hPa)')
        cs = ax.contour(lon,lat,smooth_mslp,np.arange(0,1200,4),colors='black',linewidths=1.5,linestyles='solid',transform=data_proj)
        plt.clabel(cs, fontsize=10, inline=1, inline_spacing=10, fmt="%i", rightside_up=True, use_clabeltext=True)
        ax.barbs(lon.values,lat.values,u_wind_10m*1.94384,v_wind_10m*1.94384,length=6,regrid_shape=20,pivot='middle',transform=data_proj,zorder=10)
        fig.tight_layout(rect=[0,0.01,1,0.98])
        plt.savefig('./%s_ML_LCL.png'%dt.strftime('%Y%m%d_%H'))
        plt.close()

        fig=plt.figure(figsize=(10,9))
        ax = fig.add_subplot(1,1,1,projection=proj)
        ax.add_feature(cfeature.STATES,edgecolor='black',linewidth=2,zorder=2)
        ax.add_feature(cfeature.COASTLINE,edgecolor='black',linewidth=2,zorder=2)
        ax.add_feature(COUNTIES,facecolor='none',edgecolor='gray',linewidth=1,zorder=1)
        ax.set_extent((-102,-93,44,48))
        cf = ax.pcolormesh(lon,lat,ML_LCL,transform=data_proj,cmap='jet_r')
        cf.set_clim(1000,700)
        cbar = plt.colorbar(cf,ax=ax,orientation='horizontal',pad=0.01,fraction=0.049)
        cbar.ax.set_xlabel('MIXED LAYER LCL (hPa)')
        cs = ax.contour(lon,lat,smooth_mslp,np.arange(0,1200,4),colors='black',linewidths=1.5,linestyles='solid',transform=data_proj)
        plt.clabel(cs, fontsize=10, inline=1, inline_spacing=10, fmt="%i", rightside_up=True, use_clabeltext=True)
        ax.barbs(lon.values,lat.values,u_wind_10m*1.94384,v_wind_10m*1.94384,length=6,regrid_shape=20,pivot='middle',transform=data_proj,zorder=10)
        fig.tight_layout(rect=[0,0.01,1,0.98])
        plt.savefig('./%s_zoomed_ML_LCL.png'%dt.strftime('%Y%m%d_%H'))
        plt.close()

        fig=plt.figure(figsize=(10,9))
        ax = fig.add_subplot(1,1,1,projection=proj)
        ax.add_feature(cfeature.STATES,edgecolor='black',linewidth=2,zorder=2)
        ax.add_feature(cfeature.COASTLINE,edgecolor='black',linewidth=2,zorder=2)
        ax.add_feature(COUNTIES,facecolor='none',edgecolor='gray',zorder=1)
        ax.set_extent((-108,-92,41,49))
        cf = ax.pcolormesh(lon,lat,SRH3,transform=data_proj,cmap='jet')
        cf.set_clim(0,500)
        cbar = plt.colorbar(cf,ax=ax,orientation='horizontal',pad=0.01,fraction=0.049)
        cbar.ax.set_xlabel('0-3km STORM RELATIVE HELICITY (m2/s2)')
        cs = ax.contour(lon,lat,smooth_mslp,np.arange(0,1200,4),colors='black',linewidths=1.5,linestyles='solid',transform=data_proj)
        plt.clabel(cs, fontsize=10, inline=1, inline_spacing=10, fmt="%i", rightside_up=True, use_clabeltext=True)
        ax.barbs(lon.values,lat.values,u_wind_10m*1.94384,v_wind_10m*1.94384,length=6,regrid_shape=20,pivot='middle',transform=data_proj,zorder=10)
        fig.tight_layout(rect=[0,0.01,1,0.98])
        plt.savefig('./%s_SRH3.png'%dt.strftime('%Y%m%d_%H'))
        plt.close()

        fig=plt.figure(figsize=(10,9))
        ax = fig.add_subplot(1,1,1,projection=proj)
        ax.add_feature(cfeature.STATES,edgecolor='black',linewidth=2,zorder=2)
        ax.add_feature(cfeature.COASTLINE,edgecolor='black',linewidth=2,zorder=2)
        ax.add_feature(COUNTIES,facecolor='none',edgecolor='gray',linewidth=1,zorder=1)
        ax.set_extent((-102,-93,44,48))
        cf = ax.pcolormesh(lon,lat,SRH3,transform=data_proj,cmap='jet')
        cf.set_clim(0,500)
        cbar = plt.colorbar(cf,ax=ax,orientation='horizontal',pad=0.01,fraction=0.049)
        cbar.ax.set_xlabel('0-3km STORM_RELATIVE_HELICITY (m2/s2)')
        cs = ax.contour(lon,lat,smooth_mslp,np.arange(0,1200,4),colors='black',linewidths=1.5,linestyles='solid',transform=data_proj)
        plt.clabel(cs, fontsize=10, inline=1, inline_spacing=10, fmt="%i", rightside_up=True, use_clabeltext=True)
        ax.barbs(lon.values,lat.values,u_wind_10m*1.94384,v_wind_10m*1.94384,length=6,regrid_shape=20,pivot='middle',transform=data_proj,zorder=10)
        fig.tight_layout(rect=[0,0.01,1,0.98])
        plt.savefig('./%s_zoomed_SRH3.png'%dt.strftime('%Y%m%d_%H'))
        plt.close()

In [5]:
def pixel_max_resize(img, h, w):
    source_h, source_w = img.shape
    return img.reshape(h,source_h // h, w, source_w // w).swapaxes(1,2).reshape(h,w,-1).max(axis=2)

In [6]:
def get_dataset(dt = None):
    print("\n")
    if dt == None:
        dt = datetime.utcnow() - timedelta(hours=1)
        dt = dt.replace(microsecond=0,second=0,minute=0)
    if debug > 0:
        print("Grabbing file for date:  %sZ\n"%(dt.strftime("%Y-%m-%d %H")))

    try:
        if debug > 0:
            print("Checking first THREDDS catalog...")
        base_url = 'https://www.ncei.noaa.gov/thredds/catalog/model-rap130anl/'
        cat = TDSCatalog(f'{base_url}{dt:%Y%m}/{dt:%Y%m%d}/catalog.xml')
        ds = cat.datasets.filter_time_range(dt,dt+timedelta(hours=0))[-1]
        ncss = ds.subset()
        if debug > 0:
            print("   DATE FOUND\n")
        return ncss
    except requests.exceptions.HTTPError as err:
        if debug > 0:
            print("   Date not in this catalog.  Trying next...\n")

    try:
        if debug > 0:
            print("Checking final THREDDS catalog...")
        base_url = 'https://www.ncei.noaa.gov/thredds/catalog/model-rap130anl-old/'
        cat = TDSCatalog(f'{base_url}{dt:%Y%m}/{dt:%Y%m%d}/catalog.xml')
        ds = cat.datasets.filter_time_range(dt,dt+timedelta(hours=0))[-1]
        ncss = ds.subset()
        return ncss
    except requests.exceptions.HTTPError as err:
        if debug > 0:
            print("   Date not in this catalog.  Exiting...\n")
        sys.exit()

In [7]:
def mixed_layer(this_MLCAPE,this_MLCIN,this_ML_TEMP,this_ML_DPT,this_ML_LCL,this_ML_LCL_Z,press,temp,dpt):
    while True:
        arg = queue.get()
        if arg is None:
            return
        i,j = arg

        this_MLCAPE[i,j] = np.nan
        this_MLCIN[i,j] = np.nan
        this_ML_TEMP[i,j] = np.nan
        this_ML_DPT[i,j] = np.nan
        this_ML_LCL[i,j] = np.nan
        this_ML_LCL_Z[i,j] = np.nan

        temp1,temp2 = mpcalc.mixed_layer_cape_cin(press[::-1],temp[:,i,j][::-1],dpt[:,i,j][::-1])
        this_MLCAPE[i,j] = temp1.magnitude
        this_MLCIN[i,j] = temp2.magnitude

        local_mixed_temp = mpcalc.mixed_layer(press[::-1],temp[:,i,j][::-1])[0]
        this_ML_TEMP[i,j] = local_mixed_temp.magnitude
        local_mixed_dpt = mpcalc.mixed_layer(press[::-1],dpt[:,i,j][::-1])[0]
        this_ML_DPT[i,j] = local_mixed_dpt.magnitude

        this_ML_LCL[i,j]
        local_mixed_lcl = mpcalc.lcl(press[-1],local_mixed_temp,local_mixed_dpt)[0]
        this_ML_LCL[i,j] = local_mixed_lcl.magnitude
        this_ML_LCL_Z[i,j] = mpcalc.pressure_to_height_std(local_mixed_lcl/100).magnitude

In [8]:
def shear(this_SMV_R,this_SMV_L,this_SMV_MEAN,this_SHEAR6,this_SHEAR3,this_EFF_SHEAR,this_SRH3,this_SRH1,press,u,v,hgt):
    while True:
        arg = queue.get()
        if arg is None:
            return
        i,j = arg

        this_hgt = hgt[:,i,j][::-1].values * units('meters')

        try:
            local_right_mover, local_left_mover, local_mean_wind = mpcalc.bunkers_storm_motion(press[::-1],u[:,i,j][::-1],v[:,i,j][::-1],this_hgt)
            this_SMV_R[i,j,0] = local_right_mover[0].magnitude
            this_SMV_L[i,j,0] = local_left_mover[0].magnitude
            this_SMV_MEAN[i,j,0] = local_mean_wind[0].magnitude
            this_SMV_R[i,j,1] = local_right_mover[1].magnitude
            this_SMV_L[i,j,1] = local_left_mover[1].magnitude
            this_SMV_MEAN[i,j,1] = local_mean_wind[1].magnitude
        except:
            this_SMV_R[i,j,0] = np.nan
            this_SMV_R[i,j,1] = np.nan
            this_SMV_L[i,j,0] = np.nan
            this_SMV_L[i,j,1] = np.nan
            this_SMV_MEAN[i,j,0] = np.nan
            this_SMV_MEAN[i,j,1] = np.nan

        try:
            u_shr6, v_shr6 = mpcalc.bulk_shear(press[::-1],u[:,i,j][::-1],v[:,i,j][::-1],this_hgt,depth=6000*units('meters'))
            this_SHEAR6[i,j,0] = u_shr6.magnitude
            this_SHEAR6[i,j,1] = v_shr6.magnitude
        except:
            this_SHEAR6[i,j,0] = np.nan
            this_SHEAR6[i,j,1] = np.nan

        try:
            u_shr3, v_shr3 = mpcalc.bulk_shear(press[::-1],u[:,i,j][::-1],v[:,i,j][::-1],this_hgt,depth=3000*units('meters'))
            this_SHEAR3[i,j,0] = u_shr3.magnitude
            this_SHEAR3[i,j,1] = v_shr3.magnitude
        except:
            this_SHEAR3[i,j,0] = np.nan
            this_SHEAR3[i,j,1] = np.nan

        try:
            u_eff_shr, v_eff_shr = mpcalc.bulk_shear(press[::-1],u[:,i,j][::-1],v[:,i,j][::-1],this_hgt)
            this_EFF_SHEAR[i,j,0] = u_eff_shr.magnitude
            this_EFF_SHEAR[i,j,1] = v_eff_shr.magnitude
        except:
            this_EFF_SHEAR[i,j,0] = np.nan
            this_EFF_SHEAR[i,j,1] = np.nan

        try:
            pos_srh3, neg_srh3, tot_srh3 = mpcalc.storm_relative_helicity(this_hgt,u[:,i,j][::-1],v[:,i,j][::-1],depth=3000*units('meters'),storm_u=local_mean_wind[0],storm_v=local_mean_wind[0])
            this_SRH3[i,j] = tot_srh3.magnitude
        except:
            this_SRH3[i,j] = np.nan

        try:
            pos_srh1, neg_srh1, tot_srh1 = mpcalc.storm_relative_helicity(this_hgt,u[:,i,j][::-1],v[:,i,j][::-1],depth=1000*units('meters'),storm_u=local_mean_wind[0],storm_v=local_mean_wind[0])
            this_SRH1[i,j] = tot_srh1.magnitude
        except:
            this_SRH1[i,j] = np.nan

In [10]:
if __name__ == "__main__":
    main()



Grabbing file for date:  2020-08-14 20Z

Checking first THREDDS catalog...
   DATE FOUND

Creating memory pointers for mixed-layer variables
Calculating thermodynamic values


TypeError: cannot pickle '_thread.lock' object